## Procedures
1) Read data from a stage with error handling

In [ ]:
CREATE OR REPLACE PROCEDURE DEV_ANCHOR_DB.IRN_RAW.LOAD_AUTHORS_FROM_STAGE()
RETURNS VARCHAR(16777216)
LANGUAGE SQL
EXECUTE AS OWNER
AS '
BEGIN
    -- Execute COPY INTO command to load data
    EXECUTE IMMEDIATE ''
        COPY INTO IRN_RAW.AUTHORS
        FROM (
            SELECT
                PARSE_JSON($1) AS data,         
                METADATA$FILENAME AS file_name,
                CURRENT_TIMESTAMP() AS event_time 
            FROM @DEV_ANCHOR_DB.IRN_RAW.FACTSET_API_STG/AUTHOR
        )
        FILE_FORMAT = (TYPE = ''''JSON'''')
        PATTERN = ''''.*.json''''
        ON_ERROR = ''''CONTINUE'''';
    '';
    
    RETURN ''Data successfully loaded into IRN_RAW.AUTHORS'';
    
EXCEPTION 
    WHEN OTHER THEN 
        RETURN ''Error occurred: '' || SQLERRM;
END;
';

2. Full program flow

In [ ]:
CREATE OR REPLACE PROCEDURE DEV_ANCHOR_DB.IRN_STAGING.IDENTIFIERS_FULL_LOAD_V2()
RETURNS VARCHAR
LANGUAGE SQL
EXECUTE AS CALLER
AS '
BEGIN
    -- Step 1: Log "Started" for Internal Stage → Raw
    CALL DEV_ANCHOR_DB.IRN_STAGING.LOG_PROCEDURE_EXECUTION(
        ''IDENTIFIERS_FULL_LOAD_V2'', ''IDENTIFIERS'', ''Started'', 
        NULL, NULL, ''Internal Stage → Raw'', ''Process started''
    );

    -- Step 2: Load new data into IRN_RAW.IDENTIFIERS
    COPY INTO IRN_RAW.IDENTIFIERS 
    FROM (
        SELECT 
            PARSE_JSON($1) AS data, 
            METADATA$FILENAME AS file_name, 
            CURRENT_TIMESTAMP() AS event_time 
        FROM @DEV_ANCHOR_DB.IRN_RAW.FACTSET_API_STG/IDENTIFIERS
    ) 
    FILE_FORMAT = (TYPE = ''JSON'') 
    PATTERN = ''.*.json'';

    -- Step 3: Check if there''s new data and log count
    DECLARE
        new_raw_count INTEGER;
    BEGIN
        SELECT COUNT(*) 
        INTO :new_raw_count 
        FROM IRN_RAW.IDENTIFIERS_STREAM 
        WHERE METADATA$ACTION = ''INSERT'';

        CALL DEV_ANCHOR_DB.IRN_STAGING.LOG_PROCEDURE_EXECUTION(
            ''IDENTIFIERS_FULL_LOAD_V2'', ''IDENTIFIERS'', ''Completed'', 
            :new_raw_count, 
            NULL, ''Internal Stage → Raw'', 
            CASE 
                WHEN :new_raw_count > 0 THEN ''Data successfully loaded into RAW'' 
                ELSE ''No new data found in RAW'' 
            END
        );

        -- Only proceed with staging operations if we have new data
        IF (:new_raw_count > 0) THEN
            -- Step 4: Log "Started" for Raw → Staging
            CALL DEV_ANCHOR_DB.IRN_STAGING.LOG_PROCEDURE_EXECUTION(
                ''IDENTIFIERS_FULL_LOAD_V2'', ''IDENTIFIERS'', ''Started'', 
                NULL, NULL, ''Raw → Staging'', ''Process started''
            );

            -- Step 5: Delete existing staging data ONLY if we have new data
            DELETE FROM IRN_STAGING.IDENTIFIERS;

            -- Step 6: Move new data from raw → staging
            INSERT INTO IRN_STAGING.IDENTIFIERS (
                id, cusip, entity_Id, isin, name, sedol, ticker, update_user
            ) 
            SELECT 
                f.value:query::VARCHAR AS id, 
                f.value:instrumentMetadata.cusip::VARCHAR AS cusip,
                f.value:instrumentMetadata.entityId::VARCHAR AS entity_Id,
                f.value:instrumentMetadata.isin::VARCHAR AS isin,
                f.value:instrumentMetadata.name::VARCHAR AS name,
                f.value:instrumentMetadata.sedol::VARCHAR AS sedol,
                f.value:instrumentMetadata.ticker::VARCHAR AS ticker,
                ''BATCH_LOAD'' AS update_user
            FROM IRN_RAW.IDENTIFIERS_STREAM IDENTIFIER, 
            LATERAL FLATTEN(INPUT => IDENTIFIER.data) f
            WHERE IDENTIFIER.METADATA$ACTION = ''INSERT'';

            -- Step 7: Log Completion for Raw → Staging
            CALL DEV_ANCHOR_DB.IRN_STAGING.LOG_PROCEDURE_EXECUTION(
                ''IDENTIFIERS_FULL_LOAD_V2'', ''IDENTIFIERS'', ''Completed'', 
                :new_raw_count,
                (SELECT COUNT(*) FROM IRN_STAGING.IDENTIFIERS), 
                ''Raw → Staging'', 
                ''Data successfully moved to staging''
            );
        ELSE
            -- Log that we''re skipping staging operations due to no new data
            CALL DEV_ANCHOR_DB.IRN_STAGING.LOG_PROCEDURE_EXECUTION(
                ''IDENTIFIERS_FULL_LOAD_V2'', ''IDENTIFIERS'', ''Completed'', 
                0,
                (SELECT COUNT(*) FROM IRN_STAGING.IDENTIFIERS), 
                ''Raw → Staging'', 
                ''Skipped staging operations - no new data to process''
            );
        END IF;
    END;

    RETURN ''Code Executed Successfully'';
END;
';